In [18]:
import pandas as pd

# GitHub raw URL 목록
urls = [
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/1.R1%7E2.csv",
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/2.RTM.csv",
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/3.R3%7E5.csv",
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/4.PLAY-IN.csv",
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/5.PLAYOFF.csv"
]

dfs = []

for url in urls:
    # 파일 이름만 추출 (예: Round1_Bahrain.csv → Round1_Bahrain)
    filename = url.split("/")[-1].replace(".csv", "")

    # CSV 읽기
    df = pd.read_csv(url)

    dfs.append(df)

# 모든 데이터 합치기
df_all = pd.concat(dfs, ignore_index=True)

In [19]:
# 타임라인 데이터 불러오기
import os
tl_df = pd.read_excel("../LCK_EventsData/timeline_events_combined.xlsx", engine='openpyxl', sheet_name="timeline_events",
                      dtype={'date': str})
tl_df = tl_df[['gameid','date','time','player','action','target','igt_fixed',
               'drake_normal','voidgrub','herald','nashor','drake_elder','kill','takedown','nexus']]

In [20]:
tl_df['date'] = pd.to_datetime(tl_df['date'], format='%y%m%d')

In [ ]:
# 마이크로초 단위 timestamp를 datetime 객체로 변환
# unit='us'를 사용하여 마이크로초 단위임을 지정합니다.
df_all['datetime_utc'] = pd.to_datetime(df_all['timestamp'], unit='us', errors='coerce')

# 초 이하 단위 제거
df_all['datetime_utc'] = df_all['datetime_utc'].dt.floor('s')

# UTC 시간을 KST (UTC+9)로 변환
# 먼저 UTC 타임존 정보를 추가하고, 그 다음 KST로 변환합니다.
df_all['datetime_kst'] = df_all['datetime_utc'].dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')

# 초 이하 단위 제거
df_all['datetime_kst'] = df_all['datetime_kst'].dt.floor('s')

#datetime_utc 열 제거
df_all = df_all.drop(columns=['datetime_utc', 'amount'])

df_all['date'] = pd.to_datetime(df_all['date'], format='%y%m%d')
df_all.head()

In [31]:
# df_all에 이벤트 열만 추가
event_types = ['drake_normal', 'voidgrub', 'herald', 'nashor', 'drake_elder', 'kill', 'takedown', 'nexus']
df_all[event_types] = 0

In [24]:
#df_all의 time_text 열 이름을 time으로 변경
df_all = df_all.rename(columns={'time_text': 'time'})

In [29]:
tl_df = tl_df[['gameid', 'date', 'player','action','target','igt_fixed', 'drake_normal', 'voidgrub', 'herald', 'nashor', 'drake_elder', 'kill', 'takedown', 'nexus']]
tl_df = tl_df.rename(columns={'igt_fixed':'time'})

In [33]:
df_all.to_csv("comments.csv", index=False)
tl_df.to_csv("tl_df.csv", index=False)

In [34]:
df_all.head()

,time,timestamp,author,message,format,date,datetime_kst,drake_normal,voidgrub,herald,nashor,drake_elder,kill,takedown,nexus
0,-1:54:31,1749014450918319,UCpTHr9Rn5_w9BEMheofv82w,KT 파이팅!!!,R1~2,2025-06-04,2025-06-04 14:20:50+09:00,0,0,0,0,0,0,0,0
1,-1:42:23,1749015179089550,UCuSabTEvRG-f7w6Pa80RyLg,쇼메이커는 무조건 1찍이다,R1~2,2025-06-04,2025-06-04 14:32:59+09:00,0,0,0,0,0,0,0,0
2,-1:32:52,1749015750551203,UCNInwRrJCCrVFBz-kkxVqfA,블루가어디임?,R1~2,2025-06-04,2025-06-04 14:42:30+09:00,0,0,0,0,0,0,0,0
3,-1:25:43,1749016179287640,UCC4cqWCL8LGCsQLmB8afheA,T,R1~2,2025-06-04,2025-06-04 14:49:39+09:00,0,0,0,0,0,0,0,0
4,-1:25:25,1749016197284369,UC13UxJwYop0g_YWB-wA08Uw,ㄷㄱㄷㄱ 아무나 이겨랑,R1~2,2025-06-04,2025-06-04 14:49:57+09:00,0,0,0,0,0,0,0,0


In [35]:
tl_df.head()

,gameid,date,player,action,target,time,drake_normal,voidgrub,herald,nashor,drake_elder,kill,takedown,nexus
0,65431,2025-04-02,Pyosik,voidgrubs,NaN,03:22:53,0,1,0,0,0,0,0,0
1,65431,2025-04-02,Pyosik,voidgrubs,NaN,03:23:02,0,1,0,0,0,0,0,0
2,65431,2025-04-02,Pyosik,voidgrubs,NaN,03:23:09,0,1,0,0,0,0,0,0
3,65431,2025-04-02,Rich,kill,DuDu,03:25:10,0,0,0,0,0,1,0,0
4,65431,2025-04-02,NaN,PLATE,MID,03:25:44,0,0,0,0,0,0,0,0


In [51]:
# tl_df에 kill_streak 열 추가
tl_df['kill_streak'] = 0

# 날짜별로 그룹화하여 처리
for date_key in tl_df['date_key'].unique():
    date_mask = tl_df['date_key'] == date_key
    date_df = tl_df[date_mask].copy()
    
    # kill 이벤트만 필터링
    kill_events = date_df[date_df['kill'] == 1].sort_values('event_sec')
    
    if len(kill_events) < 3:
        continue
    
    # 각 킬에 대해 10초 이내의 킬 개수 확인
    for idx in kill_events.index:
        current_time = tl_df.loc[idx, 'event_sec']
        
        # 현재 킬 기준 10초 이내의 킬들 찾기
        time_window = (kill_events['event_sec'] >= current_time) & \
                      (kill_events['event_sec'] <= current_time + 10)
        kills_in_window = time_window.sum()
        
        # 3개 이상이면 해당 킬들을 kill_streak으로 마킹
        if kills_in_window >= 3:
            window_indices = kill_events[time_window].index
            tl_df.loc[window_indices, 'kill_streak'] = 1

print(f"총 kill_streak 이벤트 수: {tl_df['kill_streak'].sum()}")

총 kill_streak 이벤트 수: 4538


In [52]:
import pandas as pd
import re

# 파일 로드
event_df = tl_df
comments_df = df_all
# 이벤트 타입 실제 컬럼명에 맞게 정의
event_types = ['drake_normal','voidgrub','herald','nashor','drake_elder','kill','takedown','nexus','kill_streak']

# HHMMSS 형식 time → 초 단위로 변환하는 함수
def time_str_to_sec(t):
    t_str = re.sub('[^0-9]', '', str(t))
    t_str = t_str.zfill(6)
    h, m, s = int(t_str[:2]), int(t_str[2:4]), int(t_str[4:])
    return h*3600 + m*60 + s

# 시간(초)로 변환
event_df['event_sec'] = event_df['time'].apply(time_str_to_sec)
comments_df['comment_sec'] = comments_df['time'].apply(time_str_to_sec)

# 날짜 key로 매칭 (동일 날짜: 같은 게임)
event_df['date_key'] = event_df['date'].astype(str)
comments_df['date_key'] = comments_df['date'].astype(str)

# 댓글 데이터에 결과 이벤트 열 추가
for etype in event_types:
    comments_df[etype + '_event'] = 0

# 댓글-이벤트 매핑 함수
def add_event_to_comments(ev_row):
    # 날짜 및 시간 범위에 해당되는 댓글 선택
    dm = comments_df['date_key'] == ev_row['date_key']
    tm = (comments_df['comment_sec'] >= ev_row['event_sec'] - 3) & (comments_df['comment_sec'] <= ev_row['event_sec'] + 10)
    for etype in event_types:
        if etype in ev_row.index and ev_row[etype] == 1:
            comments_df.loc[dm & tm, etype + '_event'] = 1

# 모든 이벤트에 대해 적용
event_df.apply(add_event_to_comments, axis=1)

# 결과 저장
comments_df.to_csv('comments_with_events.csv', index=False)


In [53]:
comments_df = comments_df[['time', 'timestamp', 'author', 'message', 'format',
       'datetime_kst', 'drake_normal_event', 'voidgrub_event', 'herald_event', 'nashor_event',
       'drake_elder_event', 'kill_event', 'takedown_event', 'nexus_event','kill_streak_event']]

In [54]:
#time에 '-' 포함된 경우 ~event 열 값을 모두 0으로 설정
comments_df.loc[comments_df['time'].str.contains('-'), [
    'drake_normal_event', 'voidgrub_event', 'herald_event', 'nashor_event',
    'drake_elder_event', 'kill_event', 'takedown_event', 'nexus_event','kill_streak_event']] = 0

In [56]:
#EDA: 이벤트별 댓글 수 확인
comments_df[['drake_normal_event', 'voidgrub_event', 'herald_event', 'nashor_event',
             'drake_elder_event', 'kill_event', 'takedown_event', 'nexus_event','kill_streak_event']].sum()

drake_normal_event    30653
voidgrub_event        14791
herald_event           6880
nashor_event          12131
drake_elder_event       849
kill_event            87582
takedown_event        26679
nexus_event           17378
kill_streak_event     21429
dtype: int64

In [57]:
comments_df.to_csv('comments_with_timeline.csv', index=False)